In [1]:
class class_constant(property):
    __slots__ = ("name",)
    
    def __get__(self, instance, owner=None):
        if instance is None:
            instance = owner
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        raise AttributeError(
            f"{self} of '{type(instance).__name__}' object cannot be modified."
        )
    
    def __delete__(self, instance):
        raise AttributeError(
            f"{self} of '{type(instance).__name__}' object cannot be deleted."
        )
    
    def __set_name__(self, owner, name):
        self.name = name
    
    def __init__(self, fget, doc = None):
        super().__init__(
            fget, fset=None, fdel=None, doc=doc or getattr(fget, "__doc__", None)
        )
        self.name = None
    
    def __repr__(self):
        return f"class constant '{self.name}'"


class MyMeta(type):
    def __new__(meta, name, bases, namespace):
        # extract user-specified constant attributes; make them class_constants
        constant_attrs = set(namespace.pop("__constant_attrs__", set()))
        for attr in constant_attrs:
            if isinstance(namespace.get(attr), class_constant):
                continue  # already handled via @-decorator
            value = namespace.pop(attr, None)
            namespace[attr] = meta._make_class_constant(value)
        
        cls = super().__new__(meta, name, bases, namespace)
        
        # add missing class_constants and those from parent classes
        for attr, val in cls.__dict__.items():
            if isinstance(val, class_constant):
                constant_attrs.add(attr)
        for base in cls.__mro__:
            constant_attrs.update(getattr(base, "__constants__", set()))
        
        cls.__constants__ = set(constant_attrs)
        return cls
    
    def __setattr__(cls, name, value):
        if cls._is_class_constant(name, cls.__mro__):
            raise AttributeError(
                f"Cannot modify class constant '{cls.__name__}.{name}'."
            )
        super().__setattr__(name, value)
    
    def __delattr__(cls, name):
        if cls._is_class_constant(name, cls.__mro__):
            raise AttributeError(
                f"Cannot delete class constant '{cls.__name__}.{name}'."
            )
        return super().__delattr__(name)
    
    @staticmethod
    def _make_class_constant(value):
        return class_constant(lambda _: value)
    
    @staticmethod
    def _is_class_constant(name, classes):
        for cls in classes:
            attr = cls.__dict__.get(name, None)
            if isinstance(attr, class_constant):
                return True
        return False


class A(metaclass=MyMeta):
    __constant_attrs__ = {"foo"}
    
    foo = 42

class B(A):
    __constant_attrs__ = {"bar"}
    
    bar = "hello there"

class C:
    pass

class D(C, B):
    __constant_attrs__ = {"gaz"}
    
    gaz = -1

d = D()

d.__constants__

{'bar', 'foo', 'gaz'}

In [2]:
d.bar

'hello there'

In [3]:
D.bar

'hello there'

In [4]:
try:
    d.bar = "hello"
except Exception as e:
    print(f"Error: {e.args[0]}")

Error: class constant 'bar' of 'D' object cannot be modified.


In [5]:
try:
    D.bar = "hello"
except Exception as e:
    print(f"Error: {e.args[0]}")

Error: Cannot modify class constant 'D.bar'.


In [6]:
try:
    setattr(D, "bar", "hello")
except Exception as e:
    print(f"Error: {e.args[0]}")

Error: Cannot modify class constant 'D.bar'.


In [7]:
d.bar

'hello there'

In [8]:
D.bar

'hello there'

In [9]:
d.__dict__

{}

In [10]:
d.__dict__["bar"] = "hello"

In [11]:
d.bar

'hello there'

In [12]:
D.gaz

-1

In [13]:
try:
    del d.gaz
except Exception as e:
    print(f"Error: {e.args[0]}")

Error: class constant 'gaz' of 'D' object cannot be deleted.


In [14]:
try:
    del D.gaz
except Exception as e:
    print(f"Error: {e.args[0]}")

Error: Cannot delete class constant 'D.gaz'.


In [15]:
try:
    del d.bar
except Exception as e:
    print(f"Error: {e.args[0]}")

Error: class constant 'bar' of 'D' object cannot be deleted.


In [16]:
try:
    del D.bar
except Exception as e:
    print(f"Error: {e.args[0]}")

Error: Cannot delete class constant 'D.bar'.
